In [1]:
import re
import numpy as np
from rsciio import bruker, emd, digitalmicrograph
from jupyterlab_h5web import H5Web
import h5py
from matplotlib import pyplot as plt
import xraydb
import flatdict as fd
from ase.data import chemical_symbols
print(chemical_symbols[1::])

['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr', 'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt', 'Ds', 'Rg', 'Cn', 'Nh', 'Fl', 'Mc', 'Lv', 'Ts', 'Og']


In [29]:
src = "/home/kaiobach/Research/paper_paper_paper/scidat_nomad_ebsd/bb_analysis/data/development_spctrscpy"
fnms = [("pdi", "InGaN_nanowires_spectra.edaxh5"),
        ("ikz", "AlGaO.nxs"),
        ("ikz", "GeSi.nxs"),
        ("ikz", "GeSn_13.nxs"),
        ("ikz", "VInP_108_L2.h5"),
        ("fhi", "CG71113 1513 HAADF-DF4-DF2-BF 1.2 Mx STEM.emd"),
        ("fhi", "CG71113 1138 Ceta 660 mm Camera.emd"),
        ("fhi", "CG71113 1125 Ceta 1.1 Mx Camera.emd"),
        ("fhi", "CG71113 1412 EDS-HAADF-DF4-DF2-BF 4.8 Mx SI.emd"),
        ("adrien", "1613_Si_HAADF_610_kx.emd"),
        ("adrien", "46_ES-LP_L1_brg.bcf"),
        ("benedikt", "EELS_map_2_ROI_1_location_4.dm3"),
        ("phillippe", "H5OINA_examples_Specimen_1_Map_EDS_+_EBSD_Map_Data_2.h5oina")]
# pyUSID, HSMA
case = 5  # 5 # len(fnms) - 1  # len(fnms) - 1
fnm = f"{src}/{fnms[case][0]}/{fnms[case][1]}"
print(fnm)

/home/kaiobach/Research/paper_paper_paper/scidat_nomad_ebsd/bb_analysis/data/development_spctrscpy/fhi/CG71113 1513 HAADF-DF4-DF2-BF 1.2 Mx STEM.emd


In [31]:
H5Web(fnm)

<jupyterlab_h5web.widget.H5Web object>

In [4]:
# resulting NeXus artifact
# tmp = "debug.CG71113 1138 Ceta 660 mm Camera.emd.nxs"
# print(tmp)
# H5Web(tmp)
H5Web(f"debug.{fnms[case][1]}.nxs")

<jupyterlab_h5web.widget.H5Web object>

## Collect unique keys from set of files

In [28]:
fnms = ["CG71113 1121 Ceta 310.0 kx Camera.emd",
        "CG71113 1125 Ceta 1.1 Mx Camera 0001.emd",
        "CG71113 1125 Ceta 1.1 Mx Camera.emd",
        "CG71113 1126 Ceta 1.1 Mx Camera.emd",
        "CG71113 1134 Ceta 660 mm Camera.emd",
        "CG71113 1138 Ceta 660 mm Camera.emd",
        "CG71113 1405 HAADF-DF4-DF2-BF 4.8 Mx STEM.emd",
        "CG71113 1407 HAADF-DF4-DF2-BF 4.8 Mx STEM.emd",
        "CG71113 1409 HAADF-DF4-DF2-BF 6.7 Mx STEM.emd",
        "CG71113 1411 HAADF-DF4-DF2-BF 4.8 Mx STEM.emd",
        "CG71113 1411 HAADF-DF4-DF2-BF 6.7 Mx STEM.emd",
        "CG71113 1412 EDS-HAADF-DF4-DF2-BF 4.8 Mx SI.emd",
        "CG71113 1422 EDS-HAADF-DF4-DF2-BF 1.2 Mx SI.emd",
        "CG71113 1423 EDS-HAADF-DF4-DF2-BF 1.2 Mx SI.emd",
        "CG71113 1444 EDS-HAADF-DF4-DF2-BF 595.5 kx SI.emd",
        "CG71113 1513 HAADF-DF4-DF2-BF 1.2 Mx STEM.emd",
        "CG71113 1514 EDS-HAADF-DF4-DF2-BF 1.2 Mx SI.emd",
        "CG71113 1537 HAADF-DF4-DF2-BF 432.2 kx STEM.emd"]

uniq = set()

for fnm in fnms:
    fpath = f"{src}/fhi/{fnm}" 
    objs = emd.file_reader(fpath)
    print(f"fpath: {fpath}, len(objs): {len(objs)}")
    for idx, obj in enumerate(objs):
        if not isinstance(obj, dict):
            raise ValueError("No dict!")
        # print(obj.keys())
        meta = fd.FlatDict(obj["metadata"], "/")
        if True is True:  # meta["General/title"] == "Te":
            # for key, val in obj.items():
            #     print(f"{idx}, {key}, {np.shape(val)}")
            # print(f"\n\n{obj['axes']}")
            # print("\n\n")
            # for key, val in meta.items():
                # print(f"{key}: {val}")
            orgmeta = fd.FlatDict(obj["original_metadata"], "/")
            # print("\n\n")
            for key, val in orgmeta.items():
                # print(f"{key}: {val}")
                if key == "Core/MetadataDefinitionVersion" or key == "Core/MetadataSchemaVersion":
                    print(f"{key}, {val}")
                uniq.add(f"{key}")
# print(f"{type(objs[0])}")
# print(objs[0].keys())
print(f"len(uniq): {len(uniq)}")

fpath: /home/kaiobach/Research/paper_paper_paper/scidat_nomad_ebsd/bb_analysis/data/development_spctrscpy/fhi/CG71113 1121 Ceta 310.0 kx Camera.emd, len(objs): 1
Core/MetadataDefinitionVersion, 7.9
Core/MetadataSchemaVersion, v1/2013/07
fpath: /home/kaiobach/Research/paper_paper_paper/scidat_nomad_ebsd/bb_analysis/data/development_spctrscpy/fhi/CG71113 1125 Ceta 1.1 Mx Camera 0001.emd, len(objs): 1
Core/MetadataDefinitionVersion, 7.9
Core/MetadataSchemaVersion, v1/2013/07
fpath: /home/kaiobach/Research/paper_paper_paper/scidat_nomad_ebsd/bb_analysis/data/development_spctrscpy/fhi/CG71113 1125 Ceta 1.1 Mx Camera.emd, len(objs): 1
Core/MetadataDefinitionVersion, 7.9
Core/MetadataSchemaVersion, v1/2013/07
fpath: /home/kaiobach/Research/paper_paper_paper/scidat_nomad_ebsd/bb_analysis/data/development_spctrscpy/fhi/CG71113 1126 Ceta 1.1 Mx Camera.emd, len(objs): 1
Core/MetadataDefinitionVersion, 7.9
Core/MetadataSchemaVersion, v1/2013/07
fpath: /home/kaiobach/Research/paper_paper_paper/scid

The file contains only one spectrum stream
The file contains only one spectrum stream


fpath: /home/kaiobach/Research/paper_paper_paper/scidat_nomad_ebsd/bb_analysis/data/development_spctrscpy/fhi/CG71113 1411 HAADF-DF4-DF2-BF 4.8 Mx STEM.emd, len(objs): 4
Core/MetadataDefinitionVersion, 7.9
Core/MetadataSchemaVersion, v1/2013/07
Core/MetadataDefinitionVersion, 7.9
Core/MetadataSchemaVersion, v1/2013/07
Core/MetadataDefinitionVersion, 7.9
Core/MetadataSchemaVersion, v1/2013/07
Core/MetadataDefinitionVersion, 7.9
Core/MetadataSchemaVersion, v1/2013/07
fpath: /home/kaiobach/Research/paper_paper_paper/scidat_nomad_ebsd/bb_analysis/data/development_spctrscpy/fhi/CG71113 1411 HAADF-DF4-DF2-BF 6.7 Mx STEM.emd, len(objs): 4
Core/MetadataDefinitionVersion, 7.9
Core/MetadataSchemaVersion, v1/2013/07
Core/MetadataDefinitionVersion, 7.9
Core/MetadataSchemaVersion, v1/2013/07
Core/MetadataDefinitionVersion, 7.9
Core/MetadataSchemaVersion, v1/2013/07
Core/MetadataDefinitionVersion, 7.9
Core/MetadataSchemaVersion, v1/2013/07
fpath: /home/kaiobach/Research/paper_paper_paper/scidat_noma

The file contains only one spectrum stream


fpath: /home/kaiobach/Research/paper_paper_paper/scidat_nomad_ebsd/bb_analysis/data/development_spctrscpy/fhi/CG71113 1444 EDS-HAADF-DF4-DF2-BF 595.5 kx SI.emd, len(objs): 10
Core/MetadataDefinitionVersion, 7.9
Core/MetadataSchemaVersion, v1/2013/07
Core/MetadataDefinitionVersion, 7.9
Core/MetadataSchemaVersion, v1/2013/07
Core/MetadataDefinitionVersion, 7.9
Core/MetadataSchemaVersion, v1/2013/07
Core/MetadataDefinitionVersion, 7.9
Core/MetadataSchemaVersion, v1/2013/07
Core/MetadataDefinitionVersion, 7.9
Core/MetadataSchemaVersion, v1/2013/07
Core/MetadataDefinitionVersion, 7.9
Core/MetadataSchemaVersion, v1/2013/07
Core/MetadataDefinitionVersion, 7.9
Core/MetadataSchemaVersion, v1/2013/07
Core/MetadataDefinitionVersion, 7.9
Core/MetadataSchemaVersion, v1/2013/07
Core/MetadataDefinitionVersion, 7.9
Core/MetadataSchemaVersion, v1/2013/07
Core/MetadataDefinitionVersion, 7.9
Core/MetadataSchemaVersion, v1/2013/07
fpath: /home/kaiobach/Research/paper_paper_paper/scidat_nomad_ebsd/bb_analy

The file contains only one spectrum stream


fpath: /home/kaiobach/Research/paper_paper_paper/scidat_nomad_ebsd/bb_analysis/data/development_spctrscpy/fhi/CG71113 1514 EDS-HAADF-DF4-DF2-BF 1.2 Mx SI.emd, len(objs): 11
Core/MetadataDefinitionVersion, 7.9
Core/MetadataSchemaVersion, v1/2013/07
Core/MetadataDefinitionVersion, 7.9
Core/MetadataSchemaVersion, v1/2013/07
Core/MetadataDefinitionVersion, 7.9
Core/MetadataSchemaVersion, v1/2013/07
Core/MetadataDefinitionVersion, 7.9
Core/MetadataSchemaVersion, v1/2013/07
Core/MetadataDefinitionVersion, 7.9
Core/MetadataSchemaVersion, v1/2013/07
Core/MetadataDefinitionVersion, 7.9
Core/MetadataSchemaVersion, v1/2013/07
Core/MetadataDefinitionVersion, 7.9
Core/MetadataSchemaVersion, v1/2013/07
Core/MetadataDefinitionVersion, 7.9
Core/MetadataSchemaVersion, v1/2013/07
Core/MetadataDefinitionVersion, 7.9
Core/MetadataSchemaVersion, v1/2013/07
Core/MetadataDefinitionVersion, 7.9
Core/MetadataSchemaVersion, v1/2013/07
Core/MetadataDefinitionVersion, 7.9
Core/MetadataSchemaVersion, v1/2013/07
fp

In [34]:
! pip list | grep rosettasciio
! pip list | grep hyperspy

rosettasciio                  0.2
hyperspy                      1.7.6


In [26]:
print(f"len(uniq): {len(uniq)}")
hashes_removed = set()
for entry in uniq:
    where = re.findall(r"([0-9a-f]{32})", entry)
    if where != []:
        if len(where) == 1:
            # print(f"{entry} >>>> {entry.replace(where[0], '*')}")
            hashes_removed.add(entry.replace(where[0], '*'))            
        else:
            raise ValueError("len(where) > 1 !")        
    else:
        hashes_removed.add(entry)
print(f"len(hashes_removed): {len(hashes_removed)}")

detectors_removed = set()
for entry in hashes_removed:
    where = re.findall(r"(Detector-[0-9]+)", entry)
    if where != []:
        if len(where) == 1:
            detectors_removed.add(entry.replace(where[0], 'Detector-*'))            
        else:
            raise ValueError("len(where) > 1 !")        
    else:
        detectors_removed.add(entry)
print(f"len(detectors_removed): {len(detectors_removed)}")

apertures_removed = set()
for entry in detectors_removed:
    where = re.findall(r"(Aperture-[0-9]+)", entry)
    if where != []:
        if len(where) == 1:
            apertures_removed.add(entry.replace(where[0], 'Aperture-*'))            
        else:
            raise ValueError("len(where) > 1 !")        
    else:
        apertures_removed.add(entry)
print(f"len(apertures_removed): {len(apertures_removed)}")

for entry in sorted(list(apertures_removed)):
    print(entry)

len(uniq): 1380
len(hashes_removed): 893
len(detectors_removed): 662
len(apertures_removed): 600
Acquisition/AcquisitionDatetime/DateTime
Acquisition/AcquisitionStartDatetime/DateTime
Acquisition/BeamType
Acquisition/SourceType
BinaryResult/AcquisitionUnit
BinaryResult/BitsPerPixel
BinaryResult/CompositionType
BinaryResult/Detector
BinaryResult/DetectorIndex
BinaryResult/Encoding
BinaryResult/ImageSize/height
BinaryResult/ImageSize/width
BinaryResult/Offset/x
BinaryResult/Offset/y
BinaryResult/PixelSize/height
BinaryResult/PixelSize/width
BinaryResult/PixelUnitX
BinaryResult/PixelUnitY
Core/MetadataDefinitionVersion
Core/MetadataSchemaVersion
Core/guid
CustomProperties/Aperture[C1].Name/type
CustomProperties/Aperture[C1].Name/value
CustomProperties/Aperture[C2].Name/type
CustomProperties/Aperture[C2].Name/value
CustomProperties/Aperture[OBJ].Name/type
CustomProperties/Aperture[OBJ].Name/value
CustomProperties/Aperture[SA].Name/type
CustomProperties/Aperture[SA].Name/value
CustomPropert

In [18]:
token = "Detectors/Detector-11/CollectionAngleRange/begin"
where = re.findall(r"(Detector-[0-9]+)", token)
print(where)

['Detector-11']
